In [12]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/happy522/ChatGPT-as-a-CAT-tool-in-Easy-language-translation_Implemenatation/refs/heads/main/GPT_4_o/Matrix_Scores_GPT4_0.csv")
df

,ID,Original,Holistic,Linguistic,TTR_Original,TTR_Holistic,TTR_Linguistic,LD_Original,LD_Holistic,LD_Linguistic,...,WSF_Linguistic,LIX_Original,LIX_Holistic,LIX_Linguistic,FRE_Original,FRE_Holistic,FRE_Linguistic,Fog_Original,Fog_Holistic,Fog_Linguistic
0,BN_10B,"Eltern, die in der Stadt Hildesheim ihren Wohn...",Eltern aus der Stadt Hildesheim können Unterst...,Eltern in Hildesheim können Geld für Babys bea...,0.601,0.585,0.606,0.508,0.532,0.590,...,3.45,55.95,31.28,29.16,33.99,61.34,65.87,17.59,8.80,9.11
1,BN_1B,BAföG (für Schüler)\n_________________________...,BAföG für Schülerinnen und Schüler\r\n\r\nWas ...,BAföG für Schülerinnen und Schüler\r\n\r\nWas ...,0.673,0.503,0.591,0.497,0.436,0.461,...,3.45,53.51,28.08,32.97,33.13,82.26,75.75,17.19,6.98,9.55
2,BN_2B,Fundsachen\n__________________________________...,Fundsachen\r\n________________________________...,Fundsachen\r\n________________________________...,0.545,0.498,0.537,0.494,0.425,0.446,...,2.31,41.74,28.82,27.44,54.91,75.13,77.85,14.23,8.63,7.09
3,BN_3B,"Kindertagesstätte, Platzvermittlung und Voranm...","Kindergärten, Platz finden und vorzeitig anmel...",Kindergärten Platzanmeldung\r\n\r\nDer Kita-Fi...,0.635,0.581,0.630,0.453,0.460,0.546,...,4.83,40.14,29.20,40.12,44.29,70.32,54.74,13.72,9.10,13.83
4,BN_4B,Allgemeine Informationen \nSie sind aus Ihrer ...,Allgemeine Informationen:\r\n- Sie ziehen aus ...,Allgemeine Informationen:\r\n\r\n- Sie ziehen ...,0.591,0.512,0.683,0.494,0.454,0.533,...,5.43,50.29,43.25,45.41,43.56,56.74,47.89,15.79,12.54,14.29
5,BN_5B,Elterngeld und Elternzeit\n___________________...,Elterngeld und Elternzeit – Erklärung in einfa...,Elterngeld ist Geld vom Staat für Eltern mit B...,0.512,0.492,0.592,0.500,0.531,0.564,...,4.62,44.24,37.92,37.41,52.47,63.63,63.38,12.57,10.48,10.97
6,BN_6B,Führungszeugnis \n____________________________...,Führungszeugnis \r\n__________________________...,Führungszeugnis: eine wichtige Urkunde\r\n\r\n...,0.487,0.525,0.614,0.499,0.507,0.540,...,5.37,47.02,40.81,47.48,44.56,59.24,49.23,14.34,10.28,12.64
7,BN_7B,Zweitwohnungssteuer \n________________________...,Zweitwohnungssteuer in einfacher Sprache:\r\n\...,Zweitwohnungssteuer: \r\n\r\nDie Stadt Braunsc...,0.581,0.502,0.752,0.480,0.482,0.515,...,4.63,44.68,36.75,40.44,45.31,70.34,59.29,14.72,10.50,13.40
8,BN_8B,Anmeldung einer Eheschliessung \nDie beabsicht...,"Ich kann helfen, den Text in leichter verständ...",**Eheschließung Anmeldung**\r\n\r\nSie müssen ...,0.547,0.534,0.588,0.519,0.446,0.486,...,4.30,51.98,38.95,38.08,42.12,60.06,61.90,18.07,12.79,11.29
9,BN_9B,Eintragung in das Wählerverzeichnis zur Landta...,Eintragung in das Wählerverzeichnis zur Landta...,Eintragung ins Wählerverzeichnis und Landtagsw...,0.407,0.429,0.636,0.458,0.395,0.505,...,5.01,48.24,41.71,46.55,52.51,57.31,52.58,12.29,10.44,11.14


In [13]:
import pandas as pd
import numpy as np
from scipy import stats

# Define which metrics are higher-is-better
metrics_info = {
    'TTR': False,
    'LD': False,
    'WR': False,
    'MDD': False,
    'SI': False,
    'Cross': False,
    'WSF': False,
    'LIX': False,
    'FRE': True,
    'Fog': False
}

def compare_columns(col1, col2):
    """
    Compare two paired sets of values and compute:
    - Shapiro-Wilk p-value for normality of differences
    - Paired t-test or Wilcoxon test
    - Effect size (Cohen's d for t-test, rank-biserial for Wilcoxon)
    """
    diff = col1 - col2
    shapiro_p = stats.shapiro(diff).pvalue

    if shapiro_p > 0.05:
        # Normal differences → paired t-test
        p_val = stats.ttest_rel(col1, col2).pvalue
        effect = diff.mean() / np.std(diff, ddof=1)  # Cohen's d
        test = "t-test"
    else:
        # Non-normal differences → Wilcoxon signed-rank test
        p_val = stats.wilcoxon(col1, col2).pvalue
        effect = (np.sum(diff > 0) - np.sum(diff < 0)) / len(diff)  # Rank-biserial correlation
        test = "Wilcoxon"

    return shapiro_p, p_val, effect, test

# Prepare results
results = []

for metric, higher_better in metrics_info.items():
    hol = df[f"{metric}_Holistic"]
    lin = df[f"{metric}_Linguistic"]
    org = df[f"{metric}_Original"]

    shapiro_p, p_val, effect, test = compare_columns(hol, lin)

    # Determine which column is better based on direction
    if higher_better:
        winner = "Holistic" if hol.mean() > lin.mean() else "Linguistic"
    else:
        winner = "Holistic" if hol.mean() < lin.mean() else "Linguistic"

    results.append({
        "Metric": metric,
        "Original_mean": round(org.mean(), 3),
        "Holistic_mean": round(hol.mean(), 3),
        "Linguistic_mean": round(lin.mean(), 3),
        "Shapiro_p": round(shapiro_p, 4),
        "p_value": round(p_val, 4),
        "Effect_size": round(effect, 2),
        "Test": test,
        "Better": winner
    })

# Convert to DataFrame and save
results_df = pd.DataFrame(results)
results_df.to_csv("Holistic_vs_Linguistic_4_0.csv", index=False)
results_df


,Metric,Original_mean,Holistic_mean,Linguistic_mean,Shapiro_p,p_value,Effect_size,Test,Better
0,TTR,0.555,0.545,0.624,0.1640,0.0001,-1.11,t-test,Holistic
1,LD,0.504,0.483,0.526,0.2674,0.0001,-1.16,t-test,Holistic
2,WR,0.492,0.437,0.492,0.0447,0.0001,-0.90,Wilcoxon,Holistic
3,MDD,2.751,2.420,2.083,0.2769,0.0000,1.17,t-test,Linguistic
4,SI,0.301,0.333,0.253,0.4184,0.0009,0.88,t-test,Linguistic
5,Cross,4.450,3.150,1.400,0.0235,0.0121,0.60,Wilcoxon,Linguistic
6,WSF,6.231,4.361,4.549,0.6773,0.3559,-0.21,t-test,Holistic
7,LIX,50.108,38.398,40.344,0.2342,0.1162,-0.37,t-test,Holistic
8,FRE,39.837,60.484,58.070,0.7705,0.2086,0.29,t-test,Holistic
9,Fog,15.954,11.215,11.988,0.9890,0.1222,-0.36,t-test,Holistic
